# Wifi experiments under ESP32 SoCs

## Installation

#### Python

#### MicroPython

#### Jupyter
You may use either JupyterLab or JupyterNotebook at your convenience

Up to run MicroPython code under ESP32, a specific Jupyter Kernel is needed

Now run either JupyterLab or JupyterNotebook 

## Experiments

### ESP-NOW communication between two cards

The Master card will be connected to the Internet by Wifi and to the Slave by ESP-NOW, and has two roles :
- it will retrieve data from a server and send it to the Slave
- it will receive data from the Slave and send it to the server

The Slave card will be connected to the Master by ESP-NOW, and has two roles:
- it will receive data from the Master (and will process actions in real life)
- it will send data (from sensors in real life) to the Master


#### Install a Micropython firmware with ESP-NOW integration

You can find informations about this special firmware here :

https://micropython-glenn20.readthedocs.io/en/latest/library/espnow.html

Run this command in a MicroPython Notebook to write the firmware to the both serial ports :

In [ ]:
%esptool --port=MASTER_PORT esp32 esp-now.bin
%esptool --port=SLAVE_PORT esp32 esp-now.bin


#### Connect to a serial port

Run this command in a MicroPython Notebook to connect to the choosed serial port :

In [ ]:
%serialconnect --port=MASTER_PORT --baud=115200

#### Enable a Wifi manager on the Master device

The serial port connected need to be Master's.

Add the Wifi Manager Module to the device. 

In [ ]:
%mpy-cross --set-exe mpy-cross

# The mpy-cross command is used to create byte code from a py file
%mpy-cross wifi_manager.py
# --binary is needed when sending a .mpy file
%sendtofile --binary --source ./wifi_manager.mpy /
%sendtofile --source ./boot_wifi_manager.py /boot.py

import machine
machine.reset()

The Wifi manager will create a Wifi Hotspot if it can't connect anywhere.

Then you need to connect a device to this Hotspot and access from a web browser to the address 192.168.4.1 and follow instructions.

#### Initialize ESP-NOW on both devices

Master

In [ ]:
%serialconnect --port=MASTER_PORT --baud=115200

%mpy-cross thingspeak.py
%sendtofile --binary --source ./thingspeak.mpy /

import network
from esp import espnow
from thinkspeak import post_thingspeak, get_thingspeak

wlan = network.WLAN(network.AP_IF)
wlan.active(True)
wlan.config(hidden=True)

# Prints Master's MAC
print(wlan.config('mac'))

Slave

In [ ]:
%serialconnect --port=SLAVE_PORT --baud=115200

import network
from esp import espnow

wlan = network.WLAN(network.AP_IF)
wlan.config(hidden=True)
wlan.config(ps_mode=network.WIFI_PS_NONE)
wlan.active(True)

# Prints Slave's MAC
print(wlan.config('mac'))

#### Make Master and Slave peers

Master

In [ ]:
%serialconnect --port=MASTER_PORT --baud=115200

e = espnow.ESPNow()
e.init()

# Put here the MAC printed
peer = b'SLAVE_MAC'

e.add_peer(peer, ifidx=network.AP_IF)

Slave


In [ ]:
%serialconnect --port=SLAVE_PORT --baud=115200

e = espnow.ESPNow()
e.init()

# Put here the MAC printed
peer = b'MASTER_MAC'

e.add_peer(peer, ifidx=network.AP_IF)

#### Retrieve data from a server and send it to the Slave

[WARNING]: warning.png

![WARNING][WARNING] Don't run it here, only run it on master.ipynb and slave.ipynb to run both at the same moment ![WARNING][WARNING]


Master

In [ ]:
%serialconnect --port=MASTER_PORT --baud=115200

e.clear(True)
while True:
    if e.poll():
        last = None
        for host, msg in e:
            if msg and (str(msg, 'utf8') != last):
                last = str(msg, 'utf8')
                post_thingspeak("E13BHXREJ6K0V1DQ", [msg])

Slave

In [ ]:
%serialconnect --port=SLAVE_PORT --baud=115200

import time

for i in range(5):
    print(i)
    ok = e.send(peer, str(i), True)
    while not ok:
        ok = e.send(peer, str(i), True)
    time.sleep(0.2)

#### Receive data from the Slave and send it to the server

[WARNING]: warning.png

![WARNING][WARNING] Don't run it here, only run it on master.ipynb and slave.ipynb to run both at the same moment ![WARNING][WARNING]

Master

In [ ]:
%serialconnect --port=MASTER_PORT --baud=115200

import time

last = ""
while True:
    data = get_thingspeak("179")
    print(data)
    if (data != last):
        last = data
        ok = e.send(peer, data, True)
        while not ok:
            ok = e.send(peer, data, True)
    time.sleep(1)


Slave

In [ ]:
%serialconnect --port=SLAVE_PORT --baud=115200

e.clear(True)
while True:
    if e.poll():
        last = None
        for host, msg in e:
            if msg and (str(msg, 'utf8') != last):
                last = str(msg, 'utf8')
                print(msg)